<a href="https://colab.research.google.com/github/Artemiy2708/ML/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%F0%9F%90%AFE_Sound_Astashkin_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small><hr style="margin:0;background-color:silver">

# **[🏆👂Phonemes](https://www.kaggle.com/c/2722phonemes/rules)**

See [**instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running and naming your Colab notebooks.

**(Optioal) CONSENT.** If ok with sharing your Colab for educational purposes, please check the box below with "X".

<mark>[ . . ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purpose. We understand that sharing is optional and this decision will not affect our grade in any way. 

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 2722phonemes             # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 2722phonemes
100% 28.8M/28.8M [00:00<00:00, 203MB/s]


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

CPU times: user 852 ms, sys: 114 ms, total: 967 ms
Wall time: 1.46 s


In [ ]:
#df = pd.read_csv('/Users/macbookpro/Desktop/XY_Phonemes.csv', low_memory=False); df

FileNotFoundError: ignored

In [ ]:
df.info()   # observe datatypes and any missing values

In [ ]:
df['g'].unique()

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**
We used pca  as it collinearity having almost normally distributed 256 features. Normalization made as, data seems all features have normal distribution or close, i was not looking for oultiers as it is seems long work here for all features and i Don't know whether it is will work, as having immense number of features, we will possibly have huge number of ouliiers and dataset becomes much less, maube underfitting.

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**
multinomial Logistic regression , we did not try KNN and lda and Qda, as we don't know how to optimize hyperparametres (parametres in Knn and we received an overfitting  having 0.9975 having 10 neighbours , but 0.43 on kaggle)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
import numpy as np, pandas as pd
from sklearn.metrics import  accuracy_score

In [ ]:
df = pd.read_csv('XY_Phonemes.csv')
df_predict=df[df['g'].isna()]
df_predict  = df_predict.drop(columns=['g'])
df_new=df[df['g'].notnull()]      
train_X = df_new.drop('g', axis=1)
train_Y= df_new['g']   
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_Y, test_size = 0.2)
X_train=X_train.apply(lambda x: (x - np.mean(x)) /  np.std(x))
X_test = X_test.apply(lambda x: (x - np.mean(x)) /  np.std(x))
df_predict = df_predict.apply(lambda x: (x - np.mean(x)) /  np.std(x))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (257) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
reduction = PCA(n_components=100)
reduction.fit(X_train)
# we leave just 0.95 expected variance  for n_compnents =100, just to reduce time of code. in this case bound is 0.9502, higher than in the easy cycle which can work ling somehow.
variance = reduction.explained_variance_ratio_
variance_bound = 0
counter = 0
for i in variance:
    variance_bound += i
    counter += 1
    if (variance_bound > 0.95):
        break
features = counter
X_train, X_test, df_predict = pd.DataFrame(reduction.transform(X_train)).loc[:, :features], pd.DataFrame(reduction.transform(X_test)).loc[:, :features], pd.DataFrame(reduction.transform(df_predict)).loc[:, :features]

PCA(n_components=100)

In [ ]:
model = LR(multi_class='multinomial', solver='lbfgs', max_iter = 10000)
accuracy=print(model.fit(X_train, Y_train).score(X_test, Y_test)) 
accuracy

0.9374


In [ ]:
pY = pd.DataFrame(model.predict(df_predict), index=range(1,len(df_predict)+1), columns=['g'])  # ensure that labels and observations are in corresponding order
pd.DataFrame(df.g.fillna('unknown').value_counts(sort=False)).rename(columns={'g':'observed output g'}).T   # distribution of all train labels
pd.DataFrame(pY.value_counts(sort=False)).rename(columns={0:'predicted output g'}).T
ToCSV(pY, 'delete_me')

,unknown,aa,ao,dcl,sh,iy
observed output g,10000,7635,11197,8340,9674,13154


g,aa,ao,dcl,iy,sh
predicted output g,1455,2328,1655,2602,1960


In [ ]:
from google.colab import files
pY.index.name = 'id'
#new = pY.rename(columns={"Class": "g"})
pY.to_csv("new.csv")
files.download("new.csv")
#kaggle competitions submit -c jul-07-22phonemes -f submission.csv -m "Message"

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
2.https://stackoverflow.com/questions/12525722/normalize-data-in-pandas/33250288#33250288

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try smarter sampling from the training set to avoid using all observations (due to time constraint) and, instead, develop more features
1. Try feature restriction, selection, and regularization (see ESL pp.140-141)
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Try encoding and incorporating speaker ID
1. Carefully read the example in [ESL textbook, p.149](https://hastie.su.domains/ElemStatLearn/printings/ESLII_print12.pdf#page=168), check seminar Colab and lecture slides
1. Check difference in distribution between test and train data. You might consider [group k-fold](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) for testing locally or your model will memorize user-phoneme releation, instead of phoneme attributes and class.
1. Learn the domain to develop better features. See ["Penalized discriminant analysis"](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C48&q=Penalized+Discriminant+Analysis&btnG=)
1. Look for training outliers that can affect your model's test performance.
  1. Essentially, you need to find and exclude training [influential points](https://en.wikipedia.org/wiki/Influential_observation), but training outliers might be influential
  1. Brute force with [LOOCV](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#Leave-one-out_cross-validation) is slow, but robust. You might find methods to find each training point's contribution to the prediction and remove rare extreme contributors (since they may not be in the test set)
  1. You can find outliers; then evaluate their effect on model's performance
    1. You can evaluate 1D and 2D histograms to find outliers and 
    1. You can cluster observations and find outliers that don't fit the clusters (DBSCAN may be helpful)
    1. You can compress high dimensional space to 1D (with PCA, tSNE, UMAP) and then find outliers with a histogram
 
 
